In [20]:
# Importing required functionalities
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from typing_extensions import Concatenate

import pandas as pd

In [21]:
import os

os.environ["OPENAI_API_KEY"] = "add your openai api key here"
api_key = "add your openai api key here"

In [23]:
import os
import glob

arr = os.listdir()

txtfiles = []
for file in glob.glob("*.pdf"):
    txtfiles.append(file)

['42479593000160-REL06022024V01-000597167.pdf', '42479593000160-REL08012024V01-000581177.pdf', '42479593000160-REL08032024V01-000617390.pdf', 'agro conceito e evolução_jan22_.pdf', 'Artigo_ A tecnologia na agricultura - Portal Embrapa.pdf', 'Atlantico - Latin America Digital Transformation Report 2023.pdf', 'building-food-and-agriculture-businesses-for-a-green-future.pdf', 'CB-Insights_Venture-Report-Q3-2023.pdf', 'Consumer_Insight__Year_in_Review_2022_en.pdf', 'Cost_of_Living__Sustainable_Behaviours_en.pdf', 'Digitalização e sust_MEGATENDENCIAS 2030.pdf.pdf.pdf', 'Future_Consumer_2025_en.pdf', 'Global Farmer Insights 2022 - McKinsey & Company_01.2023.pdf', 'pdf1.pdf', 'pdf2.pdf', 'pdf_v2_1.pdf', 'pdf_v2_2.pdf', 'pdf_v2_3.pdf', 'pdf_v2_4.pdf', 'pdf_v2_5.pdf', 'teste_agtech.pdf']


In [24]:
class PDF:
    def __init__(self, id, text):
        self.id = id
        self.text = text
    
    def split_text(self):
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=1000,
            chunk_overlap= 200,
        )

        return(text_splitter.split_text(self.text))

In [25]:
def extract_pdf_text(file_path, id):
    pdf_file = PdfReader(file_path)
    text_data = f"----INICIO RELATORIO {id}----"
    for i in range (len(pdf_file.pages)):
        text_data += pdf_file.pages[i].extract_text()
    return PDF(id, text_data)

# paths = ["pdf_v2_1.pdf", "pdf_v2_2.pdf", "pdf_v2_3.pdf", "pdf_v2_4.pdf", "pdf_v2_5.pdf"]

# paths = ['agro conceito e evolução_jan22_.pdf', 
#          'Artigo_ A tecnologia na agricultura - Portal Embrapa.pdf'
#          ]

# paths = ["teste_agtech.pdf", 
#          #"Atlantico - Latin America Digital Transformation Report 2023.pdf", 
#          #"building-food-and-agriculture-businesses-for-a-green-future.pdf", 
#          #"CB-Insights_Venture-Report-Q3-2023.pdf",
#          #"Consumer_Insight__Year_in_Review_2022_en.pdf",
#          #"Cost_of_Living__Sustainable_Behaviours_en.pdf",
#          "Global Farmer Insights 2022 - McKinsey & Company_01.2023.pdf",
#          ] 

paths = ['42479593000160-REL06022024V01-000597167.pdf', 
         '42479593000160-REL08012024V01-000581177.pdf', 
         '42479593000160-REL08032024V01-000617390.pdf']

documents = []

for i, pdf_path in enumerate(paths):
    document = extract_pdf_text(pdf_path, id=i+1)
    documents.append(document)

In [26]:
text_sections = []

for doc in documents:
    text_sections += doc.split_text()

In [27]:
embed_tool = OpenAIEmbeddings()
vector_store = FAISS.from_texts(text_sections, embed_tool)

In [28]:
text_sections

['----INICIO RELATORIO 1----Informação Pública –Política de segurança da Informação \nItaú Asset Rural Fiagro Imobiliário\nJaneiro | 2024\nItaú Asset Rural Fiagro Imobiliário\nJaneiro | 2024\nMês de \nreferênciaData de \npagamentoValor Patrimonial \nda Cota no início \ndo período [B]Rendimento \n(R$ / Cota) [A]Rendimento \nMensal (%) \n[A/B]Rendimento \nMensal (%) [A/B] \nAnualizado\nAgo /23 08/09/2023 10,00 0,12 1,20% 15,39%\nSet/23 06/10/2023 10,05 0,12 1,19% 15,31%\nOut/23 08/11/2023 10,02 0,12 1,20% 15,36%\nNov /23 07/12/2023 10,01 0,10 1,00% 12,67%\nDez/23 08/01/2024 10,01 0,10 1,00% 12,67%\nJan/24 07/02/2024 10,01 0,11 1,10% 14,01%\n*Esta rentabilidade alvo não representa e nem deve ser considerada, sob qualquer hipótese, como promessa, garantia ou sugestão de r entabilidade\nJaneiro | 2024\nO que é?\nFundo de investimento nas cadeias \nprodutivas do agronegócio no \nBrasil (insumos agrícolas, \nprodutores rurais, agroindústrias). \nComo?\nInvestimento via títulos de crédito \npr

In [33]:
inicializador = "Sendo o início de cada relatório dado por '----INICIO RELATORIO----.' e mantendo a ordem de data de publicação."
finalizador_descricao = "Separe as informações da descrição em tópicos."
finalizador_shared_info = "Separe as informações em tópicos."
formato_descricao = "Seguindo o formato para a tabela a ser preenchida: '\nTítulo': \nData de publicação: \nTópico principal: \nDescrição detalhada: '"
formato_shared_info = "Seguindo o formato para a tabela a ser preenchida: '\nNúmero de relatórios': \nInformações em comum: \nInformações diferentes: \n '"

#query = inicializador + formato_descricao + "Descreva os 4 relatórios e aponte detalhadamente as informações em comum sobre a balança comercial." + finalizador_descricao
#query = inicializador + "Qual o número de relatórios que falam sobre a balança comercial?"
#query = inicializador + "" + finalizador_shared_info

# query = inicializador + formato_descricao + "Descreva os relatórios detalhadamente e aponte as informações em comum entre eles, levando o tópico principal de cada relatório." + finalizador_shared_info

query = inicializador + formato_descricao + "Descreva os relatórios detalhadamente e aponte informações importantes para decisões de investimentos, como crescimentos e quedas, levando o tópico principal de cada relatório." + finalizador_shared_info

docs = vector_store.similarity_search(query)

In [18]:
# docs_and_scores = vector_store.similarity_search_with_score(query)

In [42]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

file1 = open("teste.txt","a", encoding='utf-8')

llm = OpenAI(temperature=0.1, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0, model ='gpt-3.5-turbo-instruct')

chain = load_qa_chain(llm, chain_type='stuff')   
response1 = chain.run(input_documents=docs, question=query)
   
file1.write("\n")
file1.write(query)
file1.write("\n")
file1.write(response1)

print(response1)

file1.close()  


Título: Relatório 1
Data de publicação: Janeiro | 2024
Tópico principal: Fundo de investimento nas cadeias produtivas do agronegócio no Brasil
Descrição detalhada:
- O relatório apresenta informações sobre o fundo de investimento Itaú Asset Rural Fiagro Imobiliário, com foco nas cadeias produtivas do agronegócio no Brasil.
- O mês de referência é janeiro de 2024.
- O valor patrimonial da cota no início do período é de 10,00.
- O rendimento mensal é de 0,12, representando uma rentabilidade de 1,20% em relação ao valor patrimonial da cota.
- O rendimento mensal anualizado é de 15,39%.
- O relatório destaca que a rentabilidade alvo não deve ser considerada como promessa, garantia ou sugestão de rentabilidade.
- O fundo investe em títulos de crédito privados ligados ao agronegócio.
- O relatório apresenta informações sobre o crescimento do fundo, com um aumento no valor patrimonial da cota em relação ao mês anterior.
- O relatório também destaca a diversificação do fundo em diferentes reg

In [38]:
response2 = chain.run(input_documents=docs, question="Apresente os números de cotistas do fundo de cada relatório.")
print(response2)



Relatório 1 (Janeiro | 2024): 63.795 cotistas
Relatório 2 (Dezembro | 2023): Não mencionado
Relatório 3 (Fevereiro | 2024): Não mencionado
